In [7]:
#from pose_estimator_2d import openpose_estimator
from utils import smooth, vis, camera
from bvh_skeleton import openpose_skeleton, h36m_skeleton, cmu_skeleton

import cv2
import importlib
import numpy as np
import os
from pathlib import Path
from IPython.display import HTML

## Estimate 2D pose from video

In [3]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2

def draw_landmarks_on_image(rgb_image, detection_result):
    pose_landmarks_list = detection_result.pose_landmarks
    annotated_image = np.copy(rgb_image)

    # Loop through the detected poses to visualize.
    for idx in range(len(pose_landmarks_list)):
        pose_landmarks = pose_landmarks_list[idx]

        # Draw the pose landmarks.
        pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        pose_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
        ])
        solutions.drawing_utils.draw_landmarks(
            annotated_image,
            pose_landmarks_proto,
            solutions.pose.POSE_CONNECTIONS,
            solutions.drawing_styles.get_default_pose_landmarks_style())

        # Add numbers beside each landmark
        for i, landmark in enumerate(pose_landmarks):
            x = int(landmark.x * annotated_image.shape[1])
            y = int(landmark.y * annotated_image.shape[0])
            cv2.putText(annotated_image, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

    return annotated_image


In [15]:
import cv2
import mediapipe as mp
import numpy as np
import time

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Open the video file
cap = cv2.VideoCapture('trial.mp4')

# Define the MediaPipe to OpenPose joint mapping (33 to 25)
# Define the MediaPipe to OpenPose joint mapping (33 to 25)
# Note: OpenPose has only 25 keypoints, so the mapping should only go from 0 to 24
mediapipe_to_openpose = {
    0: 0,  # Nose
    1: 1,  # Left Eye (MediaPipe -> OpenPose's Left Eye)
    2: 2,  # Right Eye (MediaPipe -> OpenPose's Right Eye)
    3: 3,  # Left Ear (MediaPipe -> OpenPose's Left Ear)
    4: 4,  # Right Ear (MediaPipe -> OpenPose's Right Ear)
    5: 5,  # Left Shoulder
    6: 6,  # Right Shoulder
    7: 7,  # Left Elbow
    8: 8,  # Right Elbow
    9: 9,  # Left Wrist
    10: 10,  # Right Wrist
    11: 11,  # Left Hip
    12: 12,  # Right Hip
    13: 13,  # Left Knee
    14: 14,  # Right Knee
    15: 15,  # Left Ankle
    16: 16,  # Right Ankle
    17: 17,  # Left Heel
    18: 18,  # Right Heel
    19: 19,  # Left Foot Index
    20: 20,  # Right Foot Index
    21: 21,  # Left Thumb (can be ignored for OpenPose)
    22: 22,  # Right Thumb (can be ignored for OpenPose)
    23: 23,  # Left Little Finger (can be ignored for OpenPose)
    24: 24,  # Right Little Finger (can be ignored for OpenPose)
    # Remove any joint mappings for indices >= 25
}

# Now, process the frame and apply the mapping
keypoints_list = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    img_height = frame.shape[0]
    img_width = frame.shape[1]
    # Convert the image to RGB (MediaPipe uses RGB)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Detect pose landmarks using MediaPipe
    result = pose.process(rgb_frame)
    
    if result.pose_landmarks:
        keypoints = np.zeros((25, 3))  # OpenPose has 25 keypoints
        
        # Map MediaPipe landmarks to OpenPose 25 keypoints
        for i, landmark in enumerate(result.pose_landmarks.landmark):
            if i in mediapipe_to_openpose:
                openpose_idx = mediapipe_to_openpose[i]
                keypoints[openpose_idx] = [landmark.x, landmark.y, landmark.z]

        # Append the 25 keypoints to the keypoints list
        keypoints_list.append(keypoints)
    else:
        keypoints_list.append(None)

    # Optional: Visualize the results
    annotated_image = frame.copy()
    for landmark in result.pose_landmarks.landmark:
        x = int(landmark.x * frame.shape[1])
        y = int(landmark.y * frame.shape[0])
        cv2.circle(annotated_image, (x, y), 5, (0, 255, 0), -1)

    # Show the frame with keypoints
    cv2.imshow('Pose Detection', annotated_image)
    
    if cv2.waitKey(5) & 0xFF == 27:  # Press 'Esc' to exit
        break

cap.release()
cv2.destroyAllWindows()

# Now you have the keypoints_list that contains 25 keypoints for each frame
# You can use it as you need for further processing or saving it in a desired format

## Process 2D pose

In [16]:
# filter out failed result
keypoints_list = smooth.filter_missing_value(
    keypoints_list=keypoints_list,
    method='ignore' # interpolation method will be implemented later
)

# smooth process will be implemented later

# save 2d pose result
pose2d = np.stack(keypoints_list)[:, :, :2]
pose2d_file = Path('2d_pose.npy')
np.save(pose2d_file, pose2d)

## Visualize 2D pose

In [ ]:
from pathlib import Path

# Set the path to save the visualized images
vis_result_dir = Path('2d_pose_vis')

# Ensure the directory exists
vis_result_dir.mkdir(parents=True, exist_ok=True)

cap = cv2.VideoCapture(str('trial.mp4'))
op_skel = openpose_skeleton.OpenPoseSkeleton()

for i, keypoints in enumerate(keypoints_list):
    ret, frame = cap.read()
    if not ret:
        break
    
    # keypoint whose detect confidence under kp_thresh will not be visualized
    output_file = vis_result_dir / f'{i:04d}.png'  # Correct path joining
    vis.vis_2d_keypoints(
        keypoints=keypoints,
        img=frame,
        skeleton=op_skel,
        kp_thresh=0.4,
        output_file=str(output_file)  # Convert to string if necessary
    )

cap.release()


## Initialize 3D pose estimator (Still working on it)

In [ ]:
from pathlib import Path
import importlib
from pose_estimator_3d import estimator_3d

# Reload the module to apply changes if necessary
importlib.reload(estimator_3d)

# Use plain strings for file paths
config_file = Path('models/linear_model.yaml')
config_file = str(config_file.resolve())  # Ensure it is an absolute path
# Use plain strings for file paths
checkpoint_file = Path('models/best_64.12.pth')
checkpoint_file = str(checkpoint_file.resolve()) 

e3d = estimator_3d.Estimator3D(
    config_file=config_file,
    checkpoint_file=checkpoint_file
)



## Estimate 3D pose from 2D pose (still working on it)

In [ ]:
pose2d = np.load(pose2d_file)
pose3d = e3d.estimate(pose2d, image_width=img_width, image_height=img_height)

## Convert 3D pose from camera coordinates to world coordinates

In [11]:
pose3d=np.load("../../waving_human_openpose_3d.npy")

In [12]:
# Remove camera to world transformation if not needed
pose3d_world = pose3d  # Data is already in world coordinates

pose3d_file =  '3d_pose.npy'
np.save(pose3d_file, pose3d_world)

## Convert 3D pose to BVH

In [14]:
bvh_file = f'{"hellococo"}.bvh'
cmu_skel = cmu_skeleton.CMUSkeleton()
channels, header = cmu_skel.poses2bvh(pose3d_world, output_file=bvh_file)

In [13]:
bvh_file = f'{"helloopen"}.bvh'
open = openpose_skeleton.OpenPoseSkeleton()
channels, header = open.poses2bvh(pose3d_world, output_file=bvh_file)

In [7]:
output = 'miscs/h36m_cxk.bvh'
h36m_skel = h36m_skeleton.H36mSkeleton()
_ = h36m_skel.poses2bvh(pose3d_world, output_file=output)